Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [196]:
from random import choices,randint, random
from collections import namedtuple
import logging

import lab9_lib

In [197]:
NUM_LOCI = 1000
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20
INSTANCES = 1

In [198]:
fitness = lab9_lib.make_problem(INSTANCES)
for n in range(POPULATION_SIZE):
    ind = choices([0, 1], k=NUM_LOCI)
    # print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}" if NUM_LOCI<=10 else f"ind {n}: {fitness(ind):.2%}")

print(fitness.calls)

0


Initialize population

In [199]:
class Individual:
    def __init__(self, genome, fitness):
        self.genome = genome
        self.fitness = fitness

def initialize_population(POPULATION_SIZE, NUM_LOCI):
    population = []

    for _ in range(POPULATION_SIZE):
        genome = tuple(random.choices([0, 1], k=NUM_LOCI))
        Fitness = fitness(genome)
        individual = Individual(genome=genome, fitness=Fitness)
        population.append(individual)

    return population


Cross over and mutation functions

In [200]:
def single_point_crossover(genome1, genome2):
    point = randint(0, len(genome1))
    return genome1[:point] + genome2[point:]
    # return genome1[:point] + genome2[point:], genome2[:point] + genome1[point:]


def multiple_point_crossover(genome1, genome2, num_points):
    points = [randint(0, len(genome1)) for _ in range(num_points)]
    points.sort()

    offspring = []
    last_point = 0

    for point in points:
        offspring += genome1[last_point:point]
        genome1, genome2 = genome2, genome1  # Swap the parents
        last_point = point
    offspring += genome1[last_point:]
    return offspring


def mutate(ind: Individual) -> Individual:
    num = randint(0, NUM_LOCI - 1)
    ind.genotype[num] = not ind.genotype[num]
    return ind

def tournament(population):
    selected = random.choices(population, k=2)
    return max(selected, key=lambda candidate: candidate[1])

In [201]:


# # Example usage
# PROBLEM_SIZE = 10
# genome1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
# genome2 = [11, 12, 13, 14, 15, 16, 17, 18]

# num_points = 4
# result_multiple_cut = multiple_point_crossover(genome1, genome2, num_points)

# print("Multiple Cut Result:", result_multiple_cut)
